In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

### Loaded dataset

In [ ]:
# Function to load image datasets
def load_image_dataset(image_dir1, image_dir2, image_size=(224, 224)):
    """Loads image datasets from two directories with corresponding labels.

    Args:
        image_dir1: Path to the first directory containing images.
        image_dir2: Path to the second directory containing images.
        image_size: Tuple specifying the desired image size (width, height).

    Returns:
        A tuple containing the image data (NumPy array) and labels (NumPy array).
    """
    images = []
    labels = []

    for filename in os.listdir(image_dir1):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(image_dir1, filename)
            try:
                img = Image.open(filepath).convert("RGB").resize(image_size)
                img_array = np.array(img) / 255.0 # Normalize pixel values
                images.append(img_array)
                labels.append(0) # Label for images from the first directory
            except Exception as e:
                print(f"Error loading {filename}: {e}")

    for filename in os.listdir(image_dir2):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(image_dir2, filename)
            try:
                img = Image.open(filepath).convert("RGB").resize(image_size)
                img_array = np.array(img) / 255.0
                images.append(img_array)
                labels.append(1) # Label for images from the second directory
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                
    return np.array(images), np.array(labels)

# Function to split dataset
def split_dataset(images, labels, test_size=0.2, val_size=0.1, random_state=42):
    """Splits the image dataset into training, testing, and validation sets."""
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=random_state, stratify=labels)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_size / (1 - test_size), random_state=random_state, stratify=y_train)
    return x_train, y_train, x_val, y_val, x_test, y_test


In [ ]:
# Example usage (Replace with your actual directories)
image_dir1 = "/Users/kitiya/Works/cherry_sorter/cherry_coffee/not_good"
image_dir2 = "/Users/kitiya/Works/cherry_sorter/cherry_coffee/good"
images, labels = load_image_dataset(image_dir1, image_dir2)
x_train, y_train, x_val, y_val, x_test, y_test = split_dataset(images, labels)

# Print dataset sizes
print("Training set size:", len(x_train))
print("Validation set size:", len(x_val))
print("Testing set size:", len(x_test))

### Training model

In [ ]:
# Define the model
def create_model():
    model = keras.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),  # Input shape for RGB images
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    return model

# Define image dimensions
img_height = 224  # Example dimensions, adjust as needed
img_width = 224

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define a ModelCheckpoint callback
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model
model.fit(x_train,
          y_train,
          batch_size=16,
          epochs=10,  # Adjust number of epochs as needed
          validation_data=(x_val, y_val),
          callbacks=[checkpoint])

# Evaluate the model
_, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

**Convert to .tflite**

In [ ]:
# Save the model as .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

### Test with captured image

In [ ]:
# load image
image_size = (224, 224)
path = "/test.jpg"

# Load, convert, and resize the image
img = Image.open(path).convert("RGB").resize(image_size)
img_array = np.array(img) / 255.0

# Add a batch dimension
img_array = np.expand_dims(img_array, axis=0)
print("image size: ", img_array.shape)

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="best_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("input: \n", input_details)
print("output: \n", output_details)

# Function for making predictions
def predict_tflite(image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = np.argmax(output_data)
    return "Class 1" if prediction == 1 else "Class 0"

# Make the prediction
prediction = predict_tflite(img_array.astype(np.float32))
print("Prediction:", prediction)
